# Ungraded Lab: Introduction to Keras callbacks

<a target="_blank" href="https://colab.research.google.com/github/LuisAngelMendozaVelasco/TensorFlow-Advanced_Techniques_Specialization/blob/master/Custom_Models_Layers_and_Loss_Functions_with_TensorFlow/Week5/Labs/C1_W5_Lab_1_exploring-callbacks.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a>

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
# %load_ext tensorboard
import os
import math
import datetime
import pandas as pd
from keras import layers, Sequential, callbacks, Input

2024-08-23 00:25:12.651424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-23 00:25:12.663767: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-23 00:25:12.667719: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-23 00:25:12.676782: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# Horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

2024-08-23 00:25:14.437478: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Could not resolve hostname', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling tensorflow_datasets/horses_or_humans/3.0.0.incompleteXFXJZ0/horses_or_humans-train.tfrecord*...:   0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling tensorflow_datasets/horses_or_humans/3.0.0.incompleteXFXJZ0/horses_or_humans-test.tfrecord*...:   0%…

Dataset horses_or_humans downloaded and prepared to ./tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


2024-08-23 00:25:38.401652: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1797 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    
    return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

2024-08-23 00:25:38.891440: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-08-23 00:25:38.895269: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
    model = Sequential([Input(shape=input_shape),
                        layers.Conv2D(16, (3, 3), activation='relu'),
                        layers.MaxPooling2D(2, 2),
                        layers.Conv2D(32, (3, 3), activation='relu'),
                        layers.MaxPooling2D(2, 2),
                        layers.Conv2D(64, (3, 3), activation='relu'),
                        layers.MaxPooling2D(2, 2),
                        layers.Flatten(),
                        layers.Dense(dense_units, activation='relu'),
                        layers.Dense(2, activation='softmax')])
    
    return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
logdir = os.path.join("./logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10


I0000 00:00:1724394340.732708   67196 service.cc:146] XLA service 0x784674029e10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724394340.732744   67196 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-08-23 00:25:40.751580: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-23 00:25:40.838129: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
E0000 00:00:1724394342.203403   67196 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1724394342.338137   67196 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


 5/26 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6256 - loss: 0.6848

I0000 00:00:1724394345.368435   67196 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


25/26 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6085 - loss: 0.6761

E0000 00:00:1724394346.977163   67192 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1724394347.114313   67192 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1724394347.248150   67192 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1724394347.381079   67192 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 228ms/step - accuracy: 0.6091 - loss: 0.6754 - val_accuracy: 0.4439 - val_loss: 0.7053
Epoch 2/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6071 - loss: 0.6474 - val_accuracy: 0.6829 - val_loss: 0.6177
Epoch 3/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6891 - loss: 0.6182 - val_accuracy: 0.8195 - val_loss: 0.5638
Epoch 4/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7456 - loss: 0.5622 - val_accuracy: 0.4829 - val_loss: 0.8709
Epoch 5/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7293 - loss: 0.5320 - val_accuracy: 0.7951 - val_loss: 0.4603
Epoch 6/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8200 - loss: 0.4469 - val_accuracy: 0.8927 - val_loss: 0.3741
Epoch 7/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.8484 - loss: 0.3676 - val_accuracy: 0.8927 - val_loss: 0.3792
Epoch 8/10
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9213 - loss: 0.3174 - val_accuracy: 0.7024 - val_loss: 

In [11]:
# %tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[callbacks.ModelCheckpoint('./weights.{epoch:02d}-{val_loss:.2f}.keras', verbose=1)])

Epoch 1/5

Epoch 1: saving model to ./weights.01-0.65.keras
26/26 - 3s - 111ms/step - accuracy: 0.5560 - loss: 0.6714 - val_accuracy: 0.7366 - val_loss: 0.6479
Epoch 2/5

Epoch 2: saving model to ./weights.02-0.61.keras
26/26 - 1s - 20ms/step - accuracy: 0.6582 - loss: 0.6300 - val_accuracy: 0.7268 - val_loss: 0.6077
Epoch 3/5

Epoch 3: saving model to ./weights.03-0.54.keras
26/26 - 1s - 22ms/step - accuracy: 0.7676 - loss: 0.5647 - val_accuracy: 0.6927 - val_loss: 0.5433
Epoch 4/5

Epoch 4: saving model to ./weights.04-0.60.keras
26/26 - 1s - 20ms/step - accuracy: 0.7822 - loss: 0.5047 - val_accuracy: 0.6049 - val_loss: 0.6034
Epoch 5/5

Epoch 5: saving model to ./weights.05-0.67.keras
26/26 - 1s - 22ms/step - accuracy: 0.8054 - loss: 0.4547 - val_accuracy: 0.5951 - val_loss: 0.6656


In [13]:
model = build_model(dense_units=256)
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[callbacks.ModelCheckpoint('./saved_model.keras', verbose=1)])


Epoch 1: saving model to ./saved_model.keras
26/26 - 3s - 105ms/step - accuracy: 0.5815 - loss: 0.6632 - val_accuracy: 0.6098 - val_loss: 0.6247


In [14]:
model = build_model(dense_units=256)
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[callbacks.ModelCheckpoint('./model.keras', verbose=1)])

Epoch 1/2

Epoch 1: saving model to ./model.keras
26/26 - 3s - 111ms/step - accuracy: 0.5900 - loss: 0.6684 - val_accuracy: 0.6585 - val_loss: 0.6350
Epoch 2/2

Epoch 2: saving model to ./model.keras
26/26 - 1s - 21ms/step - accuracy: 0.7080 - loss: 0.6096 - val_accuracy: 0.7512 - val_loss: 0.5719


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[callbacks.EarlyStopping(patience=3,
                                             min_delta=0.05,
                                             baseline=0.8,
                                             mode='min',
                                             monitor='val_loss',
                                             restore_best_weights=True,
                                             verbose=1)])

Epoch 1/50
26/26 - 3s - 103ms/step - accuracy: 0.6350 - loss: 0.6589 - val_accuracy: 0.8098 - val_loss: 0.6301
Epoch 2/50
26/26 - 0s - 19ms/step - accuracy: 0.7141 - loss: 0.6013 - val_accuracy: 0.7707 - val_loss: 0.5655
Epoch 3/50
26/26 - 1s - 19ms/step - accuracy: 0.7713 - loss: 0.5359 - val_accuracy: 0.8195 - val_loss: 0.5120
Epoch 4/50
26/26 - 0s - 19ms/step - accuracy: 0.8029 - loss: 0.4624 - val_accuracy: 0.8244 - val_loss: 0.4694
Epoch 5/50
26/26 - 0s - 19ms/step - accuracy: 0.8102 - loss: 0.4209 - val_accuracy: 0.8976 - val_loss: 0.3832
Epoch 6/50
26/26 - 0s - 19ms/step - accuracy: 0.8869 - loss: 0.3369 - val_accuracy: 0.9268 - val_loss: 0.2835
Epoch 7/50
26/26 - 0s - 19ms/step - accuracy: 0.9051 - loss: 0.2821 - val_accuracy: 0.9366 - val_loss: 0.2240
Epoch 8/50
26/26 - 1s - 20ms/step - accuracy: 0.9258 - loss: 0.2345 - val_accuracy: 0.9854 - val_loss: 0.1709
Epoch 9/50
26/26 - 0s - 18ms/step - accuracy: 0.9465 - loss: 0.1947 - val_accuracy: 0.9220 - val_loss: 0.2024
Epoch 10/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
csv_file = './training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[callbacks.CSVLogger(csv_file)])

Epoch 1/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - accuracy: 0.5318 - loss: 0.6802 - val_accuracy: 0.5415 - val_loss: 0.6577
Epoch 2/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6431 - loss: 0.6379 - val_accuracy: 0.7317 - val_loss: 0.6009
Epoch 3/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7146 - loss: 0.5749 - val_accuracy: 0.8244 - val_loss: 0.5233
Epoch 4/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7880 - loss: 0.5143 - val_accuracy: 0.6098 - val_loss: 0.5943
Epoch 5/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8162 - loss: 0.4627 - val_accuracy: 0.8195 - val_loss: 0.4501


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.583942,0.669105,0.541463,0.657668
1,1,0.666667,0.626134,0.731707,0.600880
2,2,0.745742,0.559699,0.824390,0.523311
3,3,0.787105,0.498727,0.609756,0.594323
4,4,0.838199,0.442464,0.819512,0.450118


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(optimizer='sgd',
			  loss='sparse_categorical_crossentropy',
			  metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
	
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[callbacks.LearningRateScheduler(step_decay, verbose=1),
					 callbacks.TensorBoard(log_dir='./log_dir')])


Epoch 1: LearningRateScheduler setting learning rate to 0.005.
Epoch 1/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - accuracy: 0.5010 - loss: 0.6815 - val_accuracy: 0.7659 - val_loss: 0.6443 - learning_rate: 0.0050

Epoch 2: LearningRateScheduler setting learning rate to 0.0025.
Epoch 2/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7404 - loss: 0.6271 - val_accuracy: 0.7561 - val_loss: 0.6259 - learning_rate: 0.0025

Epoch 3: LearningRateScheduler setting learning rate to 0.00125.
Epoch 3/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7647 - loss: 0.6115 - val_accuracy: 0.7512 - val_loss: 0.6186 - learning_rate: 0.0012

Epoch 4: LearningRateScheduler setting learning rate to 0.000625.
Epoch 4/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7863 - loss: 0.5946 - val_accuracy: 0.7659 - val_loss: 0.6086 - learning_rate: 6.2500e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0003125.
Epoch 5/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.77

In [19]:
# %tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 factor=0.2, verbose=1,
                                                 patience=1, min_lr=0.001),
                     callbacks.TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - accuracy: 0.5617 - loss: 0.6685 - val_accuracy: 0.7512 - val_loss: 0.6295 - learning_rate: 0.0100
Epoch 2/50
25/26 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7047 - loss: 0.6195
Epoch 2: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7035 - loss: 0.6191 - val_accuracy: 0.5317 - val_loss: 0.6722 - learning_rate: 0.0100
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7099 - loss: 0.5698 - val_accuracy: 0.7951 - val_loss: 0.5552 - learning_rate: 0.0020
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7699 - loss: 0.5398 - val_accuracy: 0.7756 - val_loss: 0.5406 - learning_rate: 0.0020
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7827 - loss: 0.5249 - val_accuracy: 0.8244 - val_loss: 0.5153 - learning_rate: 0.0020
Epoch 6/50
25/26 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7773 - loss: 0.5136
Epoch 6: R

In [21]:
# %tensorboard --logdir log_dir

In [22]:
!rm -rf *.keras *.csv ./log_dir ./logs ./tensorflow_datasets